In [1]:
import dbcp

raw_gs_dfs = dbcp.extract.gridstatus_isoqueues.extract()

/app/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
/app/.local/lib/python3.10/site-packages/pudl/analysis/spatial.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/

In [2]:
from pathlib import Path

source_path = Path("/app/data/raw/queues_2022_clean_data.xlsx")
lbnl_raw = dbcp.extract.lbnl_iso_queue.extract(source_path)["lbnl_iso_queue"]

In [3]:
lbnl_raw.duplicated(subset=["queue_id", "region"], keep=False).value_counts()

False    25119
True      3914
dtype: int64

In [4]:
natural_key_fields = ["queue_id", "region"]

In [5]:
common_fields = None
for iso, df in raw_gs_dfs.items():
    if not common_fields:
        common_fields = set(df.columns)
    else:
        common_fields = common_fields.intersection(set(df.columns))

In [6]:
import pandas as pd

gs = []
for iso, df in raw_gs_dfs.items():
    df["region"] = iso.upper()
    gs.append(df)
gs = pd.concat(gs)

gs["Queue ID"] = gs["Queue ID"].astype("string")

In [7]:
lbnl = lbnl_raw[~lbnl_raw.region.str.contains("non-ISO").fillna(False)]
lbnl["region"] = lbnl["region"].str.replace("ISO-NE", "ISONE")
lbnl["queue_id"] = lbnl["queue_id"].astype("string")

lbnl.region.value_counts()

/tmp/ipykernel_47445/2365373187.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["region"] = lbnl["region"].str.replace("ISO-NE", "ISONE")
/tmp/ipykernel_47445/2365373187.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["queue_id"] = lbnl["queue_id"].astype("string")


PJM      7738
MISO     4071
CAISO    2274
ERCOT    1952
NYISO    1260
ISONE    1255
SPP       802
Name: region, dtype: int64

In [8]:
print(lbnl.shape)
print(gs.shape)

(19353, 29)
(21257, 116)


## Projects in GS not in LBNL
- Expect projects that enetered the queue after 2023
- Expect withdrawn projects from SPP


### Cleaning to make a fair comparison

In [9]:
lbnl["queue_date"] = pd.to_datetime(lbnl["queue_date"])

# NYISO
is_region = gs.region.eq("NYISO")
gs.loc[is_region, "Queue ID"] = gs.loc[is_region, "Queue ID"].str.zfill(4)

is_region = lbnl.region.eq("NYISO")
lbnl.loc[is_region, "queue_id"] = lbnl.loc[is_region, "queue_id"].str.zfill(4)

# SPP
# is_region = gs.region.eq("SPP")
# gs["Withdrawn Date"] = pd.to_datetime(gs["Withdrawn Date"], utc=True)
# gs = gs[~(gs["Withdrawn Date"].dt.year.lt(2023) & is_region)]

# PJM
gs["Queue ID"] = gs["Queue ID"].str.replace(" - moved to TC1", "")

lbnl["date_withdrawn"] = pd.to_datetime(lbnl["date_withdrawn"])
lbnl["date_operational"] = pd.to_datetime(lbnl["date_operational"])

/tmp/ipykernel_47445/3240595588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["queue_date"] = pd.to_datetime(lbnl["queue_date"])
/tmp/ipykernel_47445/3240595588.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["date_withdrawn"] = pd.to_datetime(lbnl["date_withdrawn"])
/tmp/ipykernel_47445/3240595588.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [10]:
lbnl["dbcp_id"] = lbnl["region"] + "-" + lbnl["queue_id"].astype("string")
gs["dbcp_id"] = gs["region"] + "-" + gs["Queue ID"].astype("string")

/tmp/ipykernel_47445/2977960012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["dbcp_id"] = lbnl["region"] + "-" + lbnl["queue_id"].astype("string")


In [11]:
print(lbnl.dbcp_id.duplicated(keep=False).value_counts())
print(gs.dbcp_id.duplicated(keep=False).value_counts())

False    19146
True       207
Name: dbcp_id, dtype: int64
False    21011
True       246
Name: dbcp_id, dtype: int64


## Projects in LBNL not in GS
- Expect some ERCOT projects that have been withdrawn or completed
- Expect projects from MISO prior to 2017


In [12]:
ids_in_lbnl_not_in_gs = set(lbnl.dbcp_id) - set(gs.dbcp_id)
print(len(ids_in_lbnl_not_in_gs))

2952


In [13]:
cap_cols = [col for col in lbnl.columns if "capacity" in col]
cap_cols

['capacity_mw_resource_1', 'capacity_mw_resource_2', 'capacity_mw_resource_3']

In [14]:
lbnl["total_capacity_mw"] = lbnl[cap_cols].sum(axis=1)

/tmp/ipykernel_47445/1998448356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lbnl["total_capacity_mw"] = lbnl[cap_cols].sum(axis=1)


In [15]:
projects_in_lbnl_not_in_gs = lbnl[lbnl.dbcp_id.isin(ids_in_lbnl_not_in_gs)]

In [16]:
not_in_gs_capacity = projects_in_lbnl_not_in_gs.groupby("region").sum()["total_capacity_mw"]
total_lbnl_capacity = lbnl.groupby("region").sum()["total_capacity_mw"]

In [17]:
not_in_gs_capacity

region
ERCOT    270789.40
MISO     343885.32
NYISO        87.40
Name: total_capacity_mw, dtype: float64

In [18]:
(not_in_gs_capacity / total_lbnl_capacity).fillna(0)

region
CAISO    0.000000
ERCOT    0.597196
ISONE    0.000000
MISO     0.455682
NYISO    0.000367
PJM      0.000000
SPP      0.000000
Name: total_capacity_mw, dtype: float64

## TODO: ERCOT

In [19]:
region = "ERCOT"
region_not_in_gs = projects_in_lbnl_not_in_gs.query("region == @region")

region_not_in_gs.queue_status.value_counts()

withdrawn      736
operational    313
active         139
Name: queue_status, dtype: int64

Hopefully these active projects are all projects that have become withdrawn or operational.

In [20]:
region_not_in_gs.query("queue_status == 'withdrawn'")["queue_date"].dt.year.value_counts(dropna=False)

2019    77
2007    73
2013    62
2018    60
2008    58
2017    50
2014    45
2006    44
2016    37
2015    31
2020    30
2009    30
2010    26
2011    26
2012    24
2021    20
2005    18
2004    10
2022     7
2002     5
2003     2
2001     1
Name: queue_date, dtype: int64

In [21]:
region_not_in_gs.query("queue_status == 'operational'")["queue_date"].dt.year.value_counts(dropna=False)

2017.0    33
2013.0    30
2006.0    24
2007.0    24
2018.0    22
2014.0    21
2012.0    21
2005.0    18
2015.0    16
2004.0    14
2019.0    14
2016.0    14
2011.0    14
2009.0    13
2008.0     9
2010.0     9
2020.0     8
2003.0     5
NaN        3
2002.0     1
Name: queue_date, dtype: int64

## MISO

In [22]:
region = "MISO"
region_not_in_gs = projects_in_lbnl_not_in_gs.query("region == @region")

region_not_in_gs.queue_status.value_counts()

withdrawn      1278
operational     415
active           64
Name: queue_status, dtype: int64

In [23]:
post_2017 = region_not_in_gs.queue_year.ge(2017)
post_2017.value_counts(normalize=True)

False    0.967558
True     0.032442
Name: queue_year, dtype: float64

In [24]:
region_not_in_gs[post_2017].queue_status.value_counts()

withdrawn      50
operational     4
active          3
Name: queue_status, dtype: int64

In [25]:
region_not_in_gs[post_2017].queue_date.dt.year.value_counts()

2021.0    48
2017.0     8
Name: queue_date, dtype: int64

- 45% of MISO capacity in LBNL does not appear in GS. 96% of the missing projects are before 2017. From MISO: "MISO recently made some updates to the interactive queue, it will only show applications associated with the MISO 2017 study cycle going forward. So, if the projects you're referring to are older projects, yes they were removed. Regarding the older queue information, this will be provided in a separate download at a later time." 
- These missing projects could be pulled from LBNL or an earlier snapshot of MISO GS data.
- There are 57 projects that are in LBNL but not in GS that entered the queue after 2017. **Does MISO sometimes remove projects?**

## NYISO

In [26]:
region = "NYISO"
region_not_in_gs = projects_in_lbnl_not_in_gs.query("region == @region")
gs_region = gs.query("region == @region")
lbnl_region = lbnl.query("region == @region")

region_not_in_gs.queue_status.value_counts()

operational    7
Name: queue_status, dtype: int64

In [27]:
# region_not_in_gs.queue_id.iloc[1]
region_not_in_gs.dbcp_id

13915    NYISO-201A
14141    NYISO-224A
14525    NYISO-201B
14912    NYISO-160A
14919    NYISO-209A
14964    NYISO-237A
15088    NYISO-127A
Name: dbcp_id, dtype: string

In [28]:
lbnl_region.queue_id.apply(len).value_counts()

4    1248
5      12
Name: queue_id, dtype: int64

In [29]:
lbnl_region[lbnl_region.queue_id.str.contains(r'[a-zA-Z]')].queue_id.apply(len).value_counts()

5    12
4     7
Name: queue_id, dtype: int64

In [30]:
gs_region.query("dbcp_id == 'NYISO-201A'")

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,facilityType,Post Generator Interconnection Agreement Status,Interconnection Approval Date,inService,giaToExec,studyCycle,studyGroup,studyPhase,svcType,dp1ErisMw,dp1NrisMw,dp2ErisMw,dp2NrisMw,sisPhase1,region,Type-1,Type-2,Type-3,Fuel-1,Fuel-2,Fuel-3,MW-1,MW-2,MW-3,Interconnection Request Receive Date,Interconnection Agreement Status,Study Process,Proposed On-line Date (as filed with IR),System Impact Study or Phase I Cluster Study,Facilities Study (FAS) or Phase II Cluster Study,Optional Study (OS),"Full Capacity, Partial or Energy Only (FC/P/EO)",Off-Peak Deliverability and Economic Only,Feasibility Study or Supplemental Review,MW In Service,Commercial Name,Initial Study,Feasibility Study,Feasibility Study Status,System Impact Study,System Impact Study Status,Facilities Study,Facilities Study Status,Interim Interconnection Service Agreement,Interim/Interconnection Service Agreement Status,Wholesale Market Participation Agreement,Construction Service Agreement,Construction Service Agreement Status,Upgrade Construction Service Agreement,Upgrade Construction Service Agreement Status,Backfeed Date,Long-Term Firm Service Start Date,Long-Term Firm Service End Date,Test Energy Date,Fuel,Technology,GIM Study Phase,Screening Study Started,Screening Study Complete,FIS Requested,FIS Approved,Economic Study Required,IA Signed,Air Permit,GHG Permit,Water Availability,Meets Planning,Meets All Planning,CDR Reporting Zone,Approved for Energization,Approved for Synchronization,Comment,In-Service Date,Commercial Operation Date,Cessation Date,Current Cluster,Cluster Group,Replacement Generator Commercial Op Date,Service Type,Status (Original),Proposed In-Service Date,Proposed Initial-Sync Date,Last Updated Date,Z,S,Availability of Studies,SGIA Tender Date,Updated,Unit,Op Date,Sync Date,Serv,I39,Dev,Zone,System Impact Study Completed,Feasiblity Study Status,Optional Interconnection Study Status,Project Status,dbcp_id


Ok all LBNL NYISO projects exist in GS. Just some queue id formatting issues.

## PJM

In [31]:
region = "PJM"
region_not_in_gs = projects_in_lbnl_not_in_gs.query("region == @region")
gs_region = gs.query("region == @region")
lbnl_region = lbnl.query("region == @region")

region_not_in_gs.queue_status.value_counts()

Series([], Name: queue_status, dtype: int64)

In [32]:
region_not_in_gs.queue_id

Series([], Name: queue_id, dtype: string)

In [33]:
gs_region["Queue ID"] = gs_region["Queue ID"].str.replace(" - moved to TC1", "")

/tmp/ipykernel_47445/2757310770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_region["Queue ID"] = gs_region["Queue ID"].str.replace(" - moved to TC1", "")


In [34]:
region_not_in_gs.queue_id.isin(gs_region["Queue ID"]).value_counts()

Series([], Name: queue_id, dtype: int64)

Yay! So all of the projects in LBNL not in GS are actually in grid status, PJM just added a weird string in the queue ID to signify the projects have been moved to TC1. What is **TC1**?

# Projects in GS not in LBNL

In [35]:
gs["Queue Date"] = pd.to_datetime(gs["Queue Date"], utc=True)

In [36]:
ids_in_gs_not_in_lbnl = set(gs.dbcp_id) - set(lbnl.dbcp_id)
print(len(ids_in_gs_not_in_lbnl))

4832


In [37]:
projects_gs_not_in_lbnl = gs[gs.dbcp_id.isin(ids_in_gs_not_in_lbnl)]

In [38]:
projects_gs_not_in_lbnl.region.value_counts()

SPP      1539
PJM      1456
ERCOT     762
NYISO     433
ISONE     399
MISO      255
CAISO       5
Name: region, dtype: int64

In [39]:
projects_gs_not_in_lbnl["Queue Date"].dt.year.value_counts(normalize=True).sort_index(ascending=False)

2024.0    0.034955
2023.0    0.340493
2022.0    0.088650
2021.0    0.042535
2020.0    0.020004
2019.0    0.022320
2018.0    0.021899
2017.0    0.037903
2016.0    0.033691
2015.0    0.034534
2014.0    0.034112
2013.0    0.030322
2012.0    0.024847
2011.0    0.021899
2010.0    0.020425
2009.0    0.030743
2008.0    0.057275
2007.0    0.040851
2006.0    0.017056
2005.0    0.008002
2004.0    0.006317
2003.0    0.005264
2002.0    0.006317
2001.0    0.012424
2000.0    0.003369
1999.0    0.001895
1998.0    0.001685
1989.0    0.000211
Name: Queue Date, dtype: float64

In [40]:
projects_gs_not_in_lbnl_pre_2023 = projects_gs_not_in_lbnl[projects_gs_not_in_lbnl["Queue Date"].dt.year.lt(2023)]
gs_projects_pre_2023 = gs[gs["Queue Date"].dt.year.lt(2023)]
projects_gs_not_in_lbnl_pre_2023.region.value_counts()

SPP      1110
PJM       881
NYISO     261
MISO      244
ERCOT     236
ISONE     233
CAISO       1
Name: region, dtype: int64

In [41]:
not_in_lbnl_capacity = projects_gs_not_in_lbnl_pre_2023.groupby("region").sum()["Capacity (MW)"]
total_gs_capacity = gs_projects_pre_2023.groupby("region").sum()["Capacity (MW)"]

In [42]:
print((not_in_lbnl_capacity / total_gs_capacity).sort_values(ascending=False))

region
SPP      0.610493
ISONE    0.320957
NYISO     0.22132
ERCOT     0.18682
PJM      0.163046
MISO     0.045623
CAISO     0.00001
Name: Capacity (MW), dtype: Float64


### SPP

In [43]:
region = "SPP"
region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")

In [44]:
region_gs["Status"].value_counts(dropna=False, normalize=True)

Withdrawn     0.98018
<NA>         0.017117
Completed    0.002703
Name: Status, dtype: Float64

In [45]:
region_gs["Queue Date"].dt.year.value_counts()

2017    129
2008    124
2016    104
2007     75
2009     71
2015     68
2006     58
2014     56
2018     53
2010     50
2011     43
2019     39
2001     33
2012     30
2013     30
2020     24
2002     22
2005     20
2004     20
2003     20
2021     19
2022     14
2000      7
1999      1
Name: Queue Date, dtype: int64

In [46]:
region_gs["Withdrawn Date"].isna().value_counts()

False    655
True     455
Name: Withdrawn Date, dtype: int64

SPP projects that are in GS but not LBNL are older withdrawn projects that don't have a withdrawn date.

### ISONE

In [47]:
region = "ISONE"
not_in_lbnl_region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")
region_gs = gs.query("region == @region")
region_lbnl = lbnl.query("region == @region")

In [48]:
region_lbnl.queue_status.value_counts(dropna=False, normalize=True)

withdrawn      0.478088
active         0.278884
operational    0.243028
Name: queue_status, dtype: float64

In [49]:
region_gs["Status"].value_counts(dropna=False, normalize=True)

Withdrawn    0.526506
Active       0.275301
Completed    0.198193
Name: Status, dtype: Float64

In [50]:
not_in_lbnl_region_gs["Status"].value_counts(dropna=False, normalize=True)

Withdrawn    0.703863
Completed    0.167382
Active       0.128755
Name: Status, dtype: Float64

In [51]:
not_in_lbnl_region_gs["Queue Date"].dt.year.value_counts()

2015    35
2022    28
2017    20
2014    20
2019    16
2001    15
2018    12
2021    11
2016    11
2013    10
2011     9
2008     8
1999     8
2020     7
2000     7
1998     7
2010     4
2006     4
2009     1
Name: Queue Date, dtype: int64

In [52]:
not_in_lbnl_region_gs["Generation Type"].value_counts(dropna=False, normalize=True)

<NA>       0.712446
WND        0.098712
SUN        0.060086
NG         0.038627
WAT        0.025751
BAT        0.021459
DFO NG     0.017167
WDS        0.008584
WND BAT    0.008584
FC         0.004292
DFO SUB    0.004292
Name: Generation Type, dtype: Float64

In [53]:
# region_gs["Generation Type"].value_counts(dropna=False, normalize=True)

In [54]:
not_in_lbnl_region_gs[not_in_lbnl_region_gs["Generation Type"].isna()]["Queue Date"].dt.year.value_counts()

2015    30
2022    17
2001    15
2014    14
2017    10
2018     9
2008     8
1999     8
2019     8
2021     8
2000     7
1998     7
2016     6
2013     5
2010     4
2006     4
2011     4
2009     1
2020     1
Name: Queue Date, dtype: int64

In [55]:
isone_cols = raw_gs_dfs["isone"].columns
not_in_lbnl_region_gs[not_in_lbnl_region_gs["Generation Type"].isna()][isone_cols]

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,Updated,Unit,Op Date,Sync Date,Serv,I39,Dev,Zone,System Impact Study Completed,Feasiblity Study Status,System Impact Study Status,Optional Interconnection Study Status,Facilities Study Status,Interconnection Agreement Status,Project Status,region
177,1333,2400 controllable ETU - unidirectional,<NA>,<NA>,CT,1. Singer 345kV Substation existing available ...,<NA>,<NA>,1100.0,<NA>,<NA>,2022-11-08 00:00:00+00:00,Withdrawn,3/31/2028,1/30/2024,<NA>,NaN,1/30/2024,<NA>,5/31/2028,3/31/2028,<NA>,N,<NA>,SWCT,N,Not Started,<NA>,<NA>,<NA>,<NA>,Under Study,ISONE
178,1332,Enable 1200 MW injection at Mystic,<NA>,<NA>,MA,<NA>,<NA>,<NA>,1200.0,<NA>,<NA>,2022-11-08 00:00:00+00:00,Withdrawn,5/1/2029,11/22/2023,<NA>,NaN,11/22/2023,<NA>,12/30/2029,5/1/2029,<NA>,N,PD,BOST,N,Document Posted,Not Started,<NA>,<NA>,<NA>,Under Study,ISONE
187,1323,Non-controllable ETU from Line 3023 substation...,<NA>,<NA>,ME,New substation on line 3023 (per QP1227 ) to C...,<NA>,<NA>,<NA>,<NA>,<NA>,2022-10-11 00:00:00+00:00,Withdrawn,10/8/2027,11/2/2022,<NA>,NaN,11/28/2022,<NA>,12/31/2027,10/8/2027,<NA>,N,PD,ME,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
188,1322,Non-controllable ETU from Line 388 substation ...,<NA>,<NA>,ME,New substation on line 388 (per QP1228 ) to Co...,<NA>,<NA>,<NA>,<NA>,<NA>,2022-10-11 00:00:00+00:00,Withdrawn,10/8/2027,11/2/2022,<NA>,NaN,11/28/2022,<NA>,12/31/2027,10/8/2027,<NA>,N,PD,ME,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
189,1321,NEP LNS,<NA>,Somerset,MA,NEP 115 kV Brayton Point,<NA>,<NA>,17.56,17.56,17.56,2022-10-06 00:00:00+00:00,Withdrawn,12/1/2024,11/30/2022,<NA>,NaN,12/1/2022,<NA>,12/1/2024,12/1/2024,<NA>,N,PD,SEMA,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,49,Irving Oil LTF PtP,<NA>,<NA>,<NA>,Import from NB Via MEPCO,<NA>,<NA>,<NA>,<NA>,<NA>,1998-09-14 00:00:00+00:00,Withdrawn,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,4/1/2001,<NA>,ES,N,<NA>,<NA>,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
1596,43,Tractebel LTF PtP,<NA>,<NA>,<NA>,Import from the New Brunswick System,<NA>,<NA>,<NA>,<NA>,<NA>,1998-06-08 00:00:00+00:00,Withdrawn,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,12/31/2002,<NA>,<NA>,N,<NA>,<NA>,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
1600,39,CVPS/GMP LTF PtP,<NA>,Clinton,NY,Import from NY via PV20,<NA>,<NA>,<NA>,<NA>,<NA>,1998-05-28 00:00:00+00:00,Withdrawn,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,12/31/2001,<NA>,MIS,N,<NA>,VT,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE
1601,39,HQ Highgate2 HVDC,<NA>,Franklin,VT,Located on the Hydro Quebec sys. N. near Highg...,<NA>,<NA>,<NA>,<NA>,<NA>,1998-05-28 00:00:00+00:00,Withdrawn,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,12/31/2001,<NA>,MIS,N,<NA>,VT,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ISONE


In [56]:
raw_isone = pd.read_excel("raw_isone.xlsx", sheet_name=0, header=4)
fields = ["Position", "Type"]
raw_isone["Position"] = raw_isone["Position"].astype("string")
not_in_lbnl_region_gs = not_in_lbnl_region_gs.merge(raw_isone[fields], how="left", left_on="Queue ID", right_on="Position")
region_gs = region_gs.merge(raw_isone[fields], how="left", left_on="Queue ID", right_on="Position")

In [57]:
region_gs["Type"].value_counts(dropna=False, normalize=False)

G      1932
ETU     220
TS       18
Name: Type, dtype: int64

In [58]:
not_in_lbnl_region_gs["Type"].value_counts(dropna=False, normalize=False)

ETU    201
G       73
TS      13
Name: Type, dtype: int64

In [59]:
not_in_lbnl_region_gs["Type"].value_counts(dropna=False, normalize=True)

ETU    0.700348
G      0.254355
TS     0.045296
Name: Type, dtype: float64

In [60]:
not_in_lbnl_region_gs.groupby("Type")["Capacity (MW)"].sum() / not_in_lbnl_region_gs["Capacity (MW)"].sum()

Type
ETU    0.989576
G      0.001044
TS      0.00938
Name: Capacity (MW), dtype: Float64

Wahoo! 99% of the capacity that is not in LBNL are "Elective Transmission Upgrade Interconnection" projects. LBNL must filter them out.

### PJM

In [61]:
region = "PJM"
not_in_lbnl_region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")
region_gs = gs.query("region == @region")
region_lbnl = lbnl.query("region == @region")

In [62]:
not_in_lbnl_region_gs["Queue Date"].dt.year.value_counts()

2008    129
2007    114
2013     82
2012     79
2014     79
2009     67
2022     50
2011     48
2010     39
2015     39
2018     29
2016     22
2017     19
2021     17
2006     14
2020     13
2005     12
2019     10
2001      8
2004      5
2002      3
2003      3
Name: Queue Date, dtype: int64

In [63]:
not_in_lbnl_region_gs["Status"].value_counts()

Withdrawn                      499
Confirmed                      182
Retracted                      121
Active                          34
In Service                      25
Annulled                        19
Engineering and Procurement      1
Name: Status, dtype: Int64

In [64]:
pjm_cols = raw_gs_dfs["pjm"].columns

In [65]:
raw_pjm = pd.read_excel("raw_pjm.xlsx", sheet_name=0)

In [66]:
fields = ["Project ID", "Project Type"]
raw_pjm["Project ID"] = raw_pjm["Project ID"].astype("string").str.replace(" - moved to TC1", "")
not_in_lbnl_region_gs = not_in_lbnl_region_gs.merge(raw_pjm[fields], how="left", left_on="Queue ID", right_on="Project ID")
region_gs = region_gs.merge(raw_pjm[fields], how="left", left_on="Queue ID", right_on="Project ID")

In [67]:
region_gs["Project Type"].value_counts()

Generation Interconnection     8184
Long-Term Firm Transmission     743
Merchant Transmission           193
Upgrade Request                  74
Name: Project Type, dtype: int64

In [68]:
not_in_lbnl_region_gs["Project Type"].value_counts()

Long-Term Firm Transmission    630
Merchant Transmission          193
Upgrade Request                 38
Generation Interconnection      20
Name: Project Type, dtype: int64

In [69]:
not_in_lbnl_region_gs["Project Type"].value_counts() / region_gs["Project Type"].value_counts()

Generation Interconnection     0.002444
Long-Term Firm Transmission    0.847914
Merchant Transmission          1.000000
Upgrade Request                0.513514
Name: Project Type, dtype: float64

In [70]:
not_in_lbnl_region_gs.groupby(["Project Type"], dropna=False)["Capacity (MW)"].sum() / not_in_lbnl_region_gs["Capacity (MW)"].sum()

Project Type
Generation Interconnection     0.022433
Long-Term Firm Transmission    0.638084
Merchant Transmission          0.322876
Upgrade Request                0.016607
Name: Capacity (MW), dtype: Float64

In [71]:
not_in_lbnl_region_gs.query("`Project Type` == 'Generation Interconnection'")["Queue Date"].value_counts()

2022-11-30 00:00:00+00:00    4
2022-12-09 00:00:00+00:00    3
2022-12-22 00:00:00+00:00    3
2022-11-10 00:00:00+00:00    2
2022-12-20 00:00:00+00:00    2
2022-08-31 00:00:00+00:00    1
2022-09-10 00:00:00+00:00    1
2022-11-23 00:00:00+00:00    1
2022-12-10 00:00:00+00:00    1
2022-12-29 00:00:00+00:00    1
2022-12-31 00:00:00+00:00    1
Name: Queue Date, dtype: int64

## NYISO

In [72]:
region = "NYISO"
not_in_lbnl_region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")
region_gs = gs.query("region == @region")
region_lbnl = lbnl.query("region == @region")

In [73]:
not_in_lbnl_region_gs["Generation Type"].value_counts()

AC Transmission    140
DC Transmission     49
Energy Storage      26
Load                16
Solar               13
Wind                 6
Methane              6
Hydro                1
Name: Generation Type, dtype: Int64

In [74]:
region_lbnl.resource_type_lbnl.value_counts()

Solar                    351
Battery                  333
Wind                     184
Gas                      119
Unknown                   86
Offshore Wind             76
Other Storage             42
Solar+Battery             41
Hydro                     17
Nuclear                    6
Other                      4
Offshore Wind+Battery      1
Name: resource_type_lbnl, dtype: int64

Looks like a bulk of projects in GS not in LBNL are transmission.

In [75]:
is_transmission = not_in_lbnl_region_gs["Generation Type"].str.contains("Transmission")
not_in_lbnl_region_gs.groupby(is_transmission)["Capacity (MW)"].sum() / not_in_lbnl_region_gs["Capacity (MW)"].sum()

Generation Type
False    0.199189
True     0.799003
Name: Capacity (MW), dtype: Float64

80% of the projects are transmission. What's going on with the remaining 30%?

In [76]:
not_transmission_gs = not_in_lbnl_region_gs[~is_transmission]

In [77]:
nyiso_cols = raw_gs_dfs["nyiso"].columns
not_transmission_gs[not_transmission_gs["Generation Type"].ne("Load")][nyiso_cols].head(10)

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,Proposed In-Service Date,Proposed Initial-Sync Date,Last Updated Date,Z,S,Availability of Studies,SGIA Tender Date,region
244,1184,NY125B - Two Rivers Solar,Boralex US Development,St-Lawrence,NY,Moses-Willis 230kV (MW2),NYPA,Solar,200.0,200.0,200.0,2021-05-20 00:00:00+00:00,Active,2025-12-01 00:00:00,<NA>,<NA>,NaT,2025-10-01,2025-11-01,2023-04-30,D,9,SRIS,NaT,NYISO
273,1239,NY Interconnection 3,NY Interconnection LLC,New York,NY,Goethals - Fresh Kills 345kV,ConEd,Energy Storage,130.3,130.3,122.0,2021-09-02 00:00:00+00:00,Active,2030-06-01 00:00:00,<NA>,<NA>,NaT,2030-03-01,2030-03-01,2023-12-31,J,P,<NA>,NaT,NYISO
346,1397,Hell Gate Energy Storage,New York Power Authority,Bronx,NY,Hell Gate Bus Section 2 - 138kV,NYPA,Energy Storage,96.0,95.0,96.0,2022-07-01 00:00:00+00:00,Active,2025-12-01 00:00:00,<NA>,<NA>,NaT,2025-12-01,2025-12-01,2023-12-31,J,P,<NA>,NaT,NYISO
347,1398,Harlem River Yards Energy Storage,New York Power Authority,Bronx,NY,Hell Gate Annex - 138kV voltage,NYPA,Energy Storage,107.0,106.0,107.0,2022-07-01 00:00:00+00:00,Active,2030-12-01 00:00:00,<NA>,<NA>,NaT,2030-12-01,2030-12-01,2023-12-31,J,P,<NA>,NaT,NYISO
348,1399,Pouch Energy Storage,New York Power Authority,Staten Island,NY,Fox Hills 138kV Substation,ConEd,Energy Storage,52.0,51.0,52.0,2022-07-01 00:00:00+00:00,Active,2025-12-01 00:00:00,<NA>,<NA>,NaT,2025-12-01,2025-12-01,2023-12-31,J,P,<NA>,NaT,NYISO
349,1401,Brentwood Energy Storage,New York Power Authority,Suffolk,NY,Brentwood Substation Bus #1 - 69kV,LIPA,Energy Storage,52.0,51.0,52.0,2022-07-01 00:00:00+00:00,Active,2025-12-01 00:00:00,<NA>,<NA>,NaT,2025-12-01,2025-12-01,2023-12-31,K,P,<NA>,NaT,NYISO
350,1402,Flynn Energy Storage,New York Power Authority,Suffolk,NY,Holtsville 138 kV substation,LIPA,Energy Storage,160.0,160.0,160.0,2022-07-01 00:00:00+00:00,Active,2030-12-01 00:00:00,<NA>,<NA>,NaT,2030-12-01,2030-12-01,2023-12-31,K,P,<NA>,NaT,NYISO
352,1404,Winner Solar,"Winner Solar, LLC",Clinton County,PA,Homer City - Mainesburg 345kV,NYSEG,Solar,120.0,120.0,120.0,2022-07-14 00:00:00+00:00,Active,2027-10-01 00:00:00,<NA>,<NA>,NaT,2027-05-01,2027-08-01,2023-04-30,C,2,<NA>,NaT,NYISO
354,1409,"Grateful Storage, LLC",Grateful Storage LLC,Orange,NY,Rock Tavern - Roseton 345kV,CHG&E,Energy Storage,250.0,250.0,250.0,2022-08-26 00:00:00+00:00,Active,2028-08-01 00:00:00,<NA>,<NA>,NaT,2028-05-01,2028-06-01,2023-06-01,G,4,<NA>,NaT,NYISO
364,1431,oyster shore III,Juno Power Management,Nassau,NY,Shore road 138 kV,LIPA,Energy Storage,300.0,300.0,300.0,2022-09-06 00:00:00+00:00,Active,2025-12-01 00:00:00,<NA>,<NA>,NaT,2025-10-01,2025-10-01,2023-04-30,K,2,<NA>,NaT,NYISO


In [78]:
region_lbnl[region_lbnl.project_name.str.contains("Oyster")]

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3,dbcp_id,total_capacity_mw
13987,1085,active,2020-08-28,2020.0,NaN,NaT,NaT,NYISO,Oyster Shore BESS,Juno Power Management LLC,LIPA,nassau,NaN,NaN,NY,NYISO,NaN,Shore Road 345kV,8/1/2023,2023.0,FES,Feasibility Study,Battery,Battery,NaN,NaN,500.0,NaN,NaN,NYISO-1085,500.0


In [79]:
not_transmission_gs["Queue Date"].dt.year.value_counts()

2022    44
2021     5
2019     3
2005     3
2020     2
2008     2
2006     2
2016     1
2018     1
2010     1
2013     1
2015     1
2004     1
2007     1
Name: Queue Date, dtype: int64

## MISO

In [80]:
region = "MISO"
not_in_lbnl_region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")
region_gs = gs.query("region == @region")
region_lbnl = lbnl.query("region == @region")

cols = raw_gs_dfs[region.lower()].columns

In [81]:
not_in_lbnl_region_gs[cols]["Generation Type"].value_counts(dropna=False)

Battery Storage    160
                    72
Wind                 5
Solar                3
High Voltage DC      3
Hybrid               1
Name: Generation Type, dtype: Int64

In [82]:
not_in_lbnl_region_gs.groupby(["Generation Type"])["Capacity (MW)"].sum() / not_in_lbnl_region_gs["Capacity (MW)"].sum()

Generation Type
                   0.169045
Battery Storage    0.673886
High Voltage DC    0.080551
Hybrid             0.009398
Solar               0.02336
Wind               0.043761
Name: Capacity (MW), dtype: Float64

In [83]:
not_in_lbnl_region_gs[cols]["Queue Date"].dt.year.value_counts()

2022    138
2021     63
2019     20
2020     19
2018      3
2017      1
Name: Queue Date, dtype: int64

In [84]:
not_in_lbnl_region_gs[cols]["Interconnection Location"].isna().value_counts()

False    244
Name: Interconnection Location, dtype: int64

In [85]:
not_in_lbnl_region_gs[cols]

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,facilityType,Post Generator Interconnection Agreement Status,Interconnection Approval Date,inService,giaToExec,studyCycle,studyGroup,studyPhase,svcType,dp1ErisMw,dp1NrisMw,dp2ErisMw,dp2NrisMw,sisPhase1,region
10,J2329,<NA>,<NA>,Vigo,IN,Wabash River-Whitesville South 230 kV,"DUKE ENERGY INDIANA, LLC",Battery Storage,100.0,100.0,100.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Battery Storage,Not Started,<NA>,2025-10-31T04:00:00+00:00,<NA>,DPP-2022,Central,Phase 1,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO
40,J2409,<NA>,<NA>,Independence,AR,INDEPENDENCE-ISES 500kV Substation,"ENTERGY ARKANSAS, LLC",Battery Storage,150.0,150.0,150.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Battery Storage,Not Started,<NA>,2026-11-01T04:00:00+00:00,<NA>,DPP-2022,South,Study Not Started,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO
42,J2458,<NA>,<NA>,Conway,AR,Pleasant Hill 500kV Substation,"ENTERGY ARKANSAS, LLC",Battery Storage,100.0,100.0,100.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Battery Storage,Not Started,<NA>,2026-11-01T04:00:00+00:00,<NA>,DPP-2022,South,Study Not Started,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO
44,J2459,<NA>,<NA>,Jefferson,AR,White Bluff EHV 500kV,"ENTERGY ARKANSAS, LLC",Battery Storage,100.0,100.0,100.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Battery Storage,Not Started,<NA>,2026-11-01T04:00:00+00:00,<NA>,DPP-2022,South,Study Not Started,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO
46,J2460,<NA>,<NA>,Jefferson,AR,White Bluff EHV 500kV,"ENTERGY ARKANSAS, LLC",Battery Storage,100.0,100.0,100.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Battery Storage,Not Started,<NA>,2026-11-01T04:00:00+00:00,<NA>,DPP-2022,South,Study Not Started,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,R1026,<NA>,<NA>,,,,,,0.0,0.0,0.0,2022-11-22 05:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,,,<NA>,<NA>,<NA>,,,,,0.0,0.0,0.0,0.0,,MISO
2557,R1027,<NA>,<NA>,,,,,,0.0,0.0,0.0,2022-11-22 05:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,,,<NA>,<NA>,<NA>,,,,,0.0,0.0,0.0,0.0,,MISO
2558,R1028,<NA>,<NA>,,,,,,0.0,0.0,0.0,2022-11-22 05:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,,,<NA>,<NA>,<NA>,,,,,0.0,0.0,0.0,0.0,,MISO
2559,R1029,<NA>,<NA>,,,,,,0.0,0.0,0.0,2022-12-12 05:00:00+00:00,Active,2024-12-01T05:00:00+00:00,<NA>,<NA>,NaN,,,<NA>,<NA>,<NA>,,,,,0.0,0.0,0.0,0.0,,MISO


In [86]:
region_lbnl[region_lbnl.point_of_interconnection.eq("INDEPENDENCE-ISES 500kV Substation")]

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3,dbcp_id,total_capacity_mw
12585,J2523,active,2022-09-20,2022.0,NaN,NaT,NaT,MISO,NaN,NaN,"Entergy Arkansas, LLC",independence,NaN,NaN,AR,MISO,NRIS,INDEPENDENCE-ISES 500kV Substation,11/1/2026,2026.0,Active,In Progress (unknown study),Solar+Battery,Solar,Battery,NaN,250.0,150.0,NaN,MISO-J2523,400.0


In [87]:
region_gs[region_gs["Queue ID"].eq("J2523")]

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,facilityType,Post Generator Interconnection Agreement Status,Interconnection Approval Date,inService,giaToExec,studyCycle,studyGroup,studyPhase,svcType,dp1ErisMw,dp1NrisMw,dp2ErisMw,dp2NrisMw,sisPhase1,region,Type-1,Type-2,Type-3,Fuel-1,Fuel-2,Fuel-3,MW-1,MW-2,MW-3,Interconnection Request Receive Date,Interconnection Agreement Status,Study Process,Proposed On-line Date (as filed with IR),System Impact Study or Phase I Cluster Study,Facilities Study (FAS) or Phase II Cluster Study,Optional Study (OS),"Full Capacity, Partial or Energy Only (FC/P/EO)",Off-Peak Deliverability and Economic Only,Feasibility Study or Supplemental Review,MW In Service,Commercial Name,Initial Study,Feasibility Study,Feasibility Study Status,System Impact Study,System Impact Study Status,Facilities Study,Facilities Study Status,Interim Interconnection Service Agreement,Interim/Interconnection Service Agreement Status,Wholesale Market Participation Agreement,Construction Service Agreement,Construction Service Agreement Status,Upgrade Construction Service Agreement,Upgrade Construction Service Agreement Status,Backfeed Date,Long-Term Firm Service Start Date,Long-Term Firm Service End Date,Test Energy Date,Fuel,Technology,GIM Study Phase,Screening Study Started,Screening Study Complete,FIS Requested,FIS Approved,Economic Study Required,IA Signed,Air Permit,GHG Permit,Water Availability,Meets Planning,Meets All Planning,CDR Reporting Zone,Approved for Energization,Approved for Synchronization,Comment,In-Service Date,Commercial Operation Date,Cessation Date,Current Cluster,Cluster Group,Replacement Generator Commercial Op Date,Service Type,Status (Original),Proposed In-Service Date,Proposed Initial-Sync Date,Last Updated Date,Z,S,Availability of Studies,SGIA Tender Date,Updated,Unit,Op Date,Sync Date,Serv,I39,Dev,Zone,System Impact Study Completed,Feasiblity Study Status,Optional Interconnection Study Status,Project Status,dbcp_id
56,J2523,<NA>,<NA>,Independence,AR,INDEPENDENCE-ISES 500kV Substation,"ENTERGY ARKANSAS, LLC",Solar,250.0,250.0,250.0,2022-09-14 04:00:00+00:00,Active,<NA>,<NA>,<NA>,NaN,Photovoltaic,Not Started,<NA>,2026-11-01T04:00:00+00:00,<NA>,DPP-2022,South,Study Not Started,Network Resource Interconnection Service (NRIS),0.0,0.0,0.0,0.0,,MISO,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaT,NaT,<NA>,NaT,<NA>,<NA>,<NA>,NaT,NaT,<NA>,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,MISO-J2523


In [88]:
region_lbnl[region_lbnl.point_of_interconnection.isin(not_in_lbnl_region_gs["Interconnection Location"])]["resource_type_lbnl"].value_counts()

Solar+Battery         102
Solar                   9
Battery                 9
Wind+Battery            4
Solar+Wind              2
Solar+Wind+Battery      1
Wind                    1
Name: resource_type_lbnl, dtype: int64

## ERCOT

In [89]:
region = "ERCOT"
not_in_lbnl_region_gs = projects_gs_not_in_lbnl_pre_2023.query("region == @region")
region_gs = gs.query("region == @region")
region_lbnl = lbnl.query("region == @region")

cols = raw_gs_dfs[region.lower()].columns

In [90]:
not_in_lbnl_region_gs["Generation Type"].value_counts()

Other - Battery Energy Storage                       199
Solar - Photovoltaic Solar                            32
Wind - Wind Turbine                                    3
Gas - Combustion (gas) Turbine, but not part o...      1
Other - Energy Storage                                 1
Name: Generation Type, dtype: Int64

In [91]:
not_in_lbnl_region_gs.groupby(["Generation Type"])["Capacity (MW)"].sum() / not_in_lbnl_region_gs["Capacity (MW)"].sum()

Generation Type
Gas - Combustion (gas) Turbine, but not part o...    0.003953
Other - Battery Energy Storage                       0.746435
Other - Energy Storage                                0.03389
Solar - Photovoltaic Solar                           0.194577
Wind - Wind Turbine                                  0.021145
Name: Capacity (MW), dtype: Float64

In [92]:
not_in_lbnl_region_gs_battery = not_in_lbnl_region_gs[not_in_lbnl_region_gs["Generation Type"].eq("Other - Battery Energy Storage")]
not_in_lbnl_region_gs_battery.head()

,Queue ID,Project Name,Interconnecting Entity,County,State,Interconnection Location,Transmission Owner,Generation Type,Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Queue Date,Status,Proposed Completion Date,Withdrawn Date,Withdrawal Comment,Actual Completion Date,facilityType,Post Generator Interconnection Agreement Status,Interconnection Approval Date,inService,giaToExec,studyCycle,studyGroup,studyPhase,svcType,dp1ErisMw,dp1NrisMw,dp2ErisMw,dp2NrisMw,sisPhase1,region,Type-1,Type-2,Type-3,Fuel-1,Fuel-2,Fuel-3,MW-1,MW-2,MW-3,Interconnection Request Receive Date,Interconnection Agreement Status,Study Process,Proposed On-line Date (as filed with IR),System Impact Study or Phase I Cluster Study,Facilities Study (FAS) or Phase II Cluster Study,Optional Study (OS),"Full Capacity, Partial or Energy Only (FC/P/EO)",Off-Peak Deliverability and Economic Only,Feasibility Study or Supplemental Review,MW In Service,Commercial Name,Initial Study,Feasibility Study,Feasibility Study Status,System Impact Study,System Impact Study Status,Facilities Study,Facilities Study Status,Interim Interconnection Service Agreement,Interim/Interconnection Service Agreement Status,Wholesale Market Participation Agreement,Construction Service Agreement,Construction Service Agreement Status,Upgrade Construction Service Agreement,Upgrade Construction Service Agreement Status,Backfeed Date,Long-Term Firm Service Start Date,Long-Term Firm Service End Date,Test Energy Date,Fuel,Technology,GIM Study Phase,Screening Study Started,Screening Study Complete,FIS Requested,FIS Approved,Economic Study Required,IA Signed,Air Permit,GHG Permit,Water Availability,Meets Planning,Meets All Planning,CDR Reporting Zone,Approved for Energization,Approved for Synchronization,Comment,In-Service Date,Commercial Operation Date,Cessation Date,Current Cluster,Cluster Group,Replacement Generator Commercial Op Date,Service Type,Status (Original),Proposed In-Service Date,Proposed Initial-Sync Date,Last Updated Date,Z,S,Availability of Studies,SGIA Tender Date,Updated,Unit,Op Date,Sync Date,Serv,I39,Dev,Zone,System Impact Study Completed,Feasiblity Study Status,Optional Interconnection Study Status,Project Status,dbcp_id
85,20INR0238,Voyager Storage,"EDF Renewables Development, Inc.",Wharton,Texas,44200 Hillje 345kV,<NA>,Other - Battery Energy Storage,50.14,NaN,NaN,2019-01-14 00:00:00+00:00,Active,2026-04-08 00:00:00,NaN,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ERCOT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Other,Battery Energy Storage,"SS Completed, FIS Started, No IA",2019-01-14,2019-03-19,2019-04-26,NaT,<NA>,NaT,Not Required,Not Required,Not Required,NaT,NaT,SOUTH,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ERCOT-20INR0238
104,21INR0027,Zier Storage,"Zier Solar, LLC",Kinney,Texas,78255 Pinto Creek 138kV,<NA>,Other - Battery Energy Storage,40.41,NaN,NaN,2018-05-24 00:00:00+00:00,Completed,2024-04-19 00:00:00,NaN,<NA>,2023-12-21 00:00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ERCOT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Other,Battery Energy Storage,"SS Completed, FIS Completed, IA",2018-05-24,2018-07-20,2018-05-24,2023-01-24,No,2020-10-22,Not Required,Not Required,Not Required,2022-09-27,2023-01-31,SOUTH,2023-11-08,2023-12-21,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaT,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ERCOT-21INR0027
105,21INR0029,Green Holly Storage,GreenGo Energy,Dawson,Texas,tap 345kV 59900 Longdraw - 79641 Farmland,<NA>,Other - Battery Energy Storage

In [93]:
region_lbnl[region_lbnl["county_1"].str.contains("hill") & region_lbnl.resource_type_lbnl.eq("Solar+Battery")]

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3,dbcp_id,total_capacity_mw
4936,19INR0169,active,2018-03-02,2018.0,3/15/2020,NaT,NaT,ERCOT,Sun Valley Solar,Sun Valley Solar LLC,NaN,hill,NaN,NaN,TX,ERCOT,NaN,tap 345kV 68090 Sam Sw - 3405 Tradinghouse,1/30/2023,2023.0,"SS Completed, FIS Completed, IA",IA Executed,Solar+Battery,Solar,Battery,NaN,2.0,101.0,NaN,ERCOT-19INR0169,103.0
5119,24INR0139,active,2021-10-15,2021.0,NaN,NaT,NaT,ERCOT,Midpoint Solar,"Midpoint Solar, LLC",NaN,hill,NaN,NaN,TX,ERCOT,NaN,138 KV Tap #3518 BLUM to #237 Blanton,8/30/2025,2025.0,"SS Completed, FIS Started, No IA",Facility Study,Solar+Battery,Solar,Battery,NaN,104.0,52.0,NaN,ERCOT-24INR0139,156.0
5164,21INR0257,active,2019-02-07,2019.0,10/18/2021,NaT,NaT,ERCOT,Mercury Solar,"Mercury Solar, LLC",NaN,hill,NaN,NaN,TX,ERCOT,NaN,68092 Liberty Station 345kV,12/31/2023,2023.0,"SS Completed, FIS Completed, IA",IA Executed,Solar+Battery,Solar,Battery,NaN,206.0,206.0,NaN,ERCOT-21INR0257,412.0
5826,21INR0379,active,2019-07-29,2019.0,11/11/2022,NaT,NaT,ERCOT,Ash Creek Solar,Ash Creek Solar LLC,NaN,hill,NaN,NaN,TX,ERCOT,NaN,68090 SAM SW 345kV,12/31/2023,2023.0,"SS Completed, FIS Completed, IA",IA Executed,Solar+Battery,Solar,Battery,NaN,409.0,309.0,NaN,ERCOT-21INR0379,718.0
6023,25INR0229,active,2022-11-11,2022.0,NaN,NaT,NaT,ERCOT,OCI Cobb Creek Solar,OCI Hillsboro Solar LLC,NaN,hill,NaN,NaN,TX,ERCOT,NaN,tap 345kV 1907 Venus - 68090 Sam Sw,6/1/2025,2025.0,"SS Started, FIS Started, No IA",System Impact Study,Solar+Battery,Solar,Battery,NaN,204.0,202.0,NaN,ERCOT-25INR0229,406.0
6366,24INR0124,active,2021-11-02,2021.0,NaN,NaT,NaT,ERCOT,High Noon Solar,"High Noon Solar Project, LLC",NaN,hill,NaN,NaN,TX,ERCOT,NaN,Tap 138kV 445 Yates - 444 Covington,12/31/2024,2024.0,"SS Completed, FIS Started, No IA",Facility Study,Solar+Battery,Solar,Battery,NaN,182.0,94.0,NaN,ERCOT-24INR0124,276.0


In [94]:
not_in_lbnl_region_gs_battery["clean_project_name"] = not_in_lbnl_region_gs_battery["Project Name"].str.split(" Storage").apply(lambda x: x[0]).str.replace(" BESS", "").str.strip()
    

not_in_lbnl_region_gs_battery["clean_project_name"].apply(lambda x: region_lbnl["project_name"].str.contains(x).any()).value_counts()

/tmp/ipykernel_47445/1072767161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_lbnl_region_gs_battery["clean_project_name"] = not_in_lbnl_region_gs_battery["Project Name"].str.split(" Storage").apply(lambda x: x[0]).str.replace(" BESS", "").str.strip()
/tmp/ipykernel_47445/1072767161.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  not_in_lbnl_region_gs_battery["clean_project_name"].apply(lambda x: region_lbnl["project_name"].str.contains(x).any()).value_counts()


True     129
False     70
Name: clean_project_name, dtype: int64

Ok using super rough project name maping, a majority of battery projects are Solar + Battery projects in LBNL.